In [4]:
!pip install azure-storage-blob
!pip install pyarrow
!pip install psycopg2 sqlalchemy
!pip install pandas
!pip install numpy
!pip install requests
!pip install azure-storage-blob
!pip install SQLAlchemy
!pip install joblib
!pip install psycopg2

In [5]:
import pandas as pd
import numpy as np
import json
import requests
from io import StringIO
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from math import ceil
import datetime
import calendar
from sqlalchemy import create_engine
import re
from joblib import Parallel, delayed

#### Grab, Upload to Azure, and Extract Data

In [6]:
# Azure Functions
def azure_upload_blob(connect_str, container_name, blob_name, file_path):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    with open(r"D:\All File\Course\CIS9440\HW1\CIS9440_HW\Citywide_Payroll_Data__Fiscal_Year__20240318.csv","rb") as data:
        blob_client.upload_blob(data, overwrite=True)
    print(f"Uploaded to Azure Blob: {blob_name}")

def azure_download_blob(connect_str, container_name, blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    download_stream = blob_client.download_blob()
    return download_stream.readall()

In [ ]:
# #grab data set since API only able to grab 1000 row, I will download it and upload it to Azure
# url = "https://data.cityofnewyork.us/resource/k397-673e.json"
# response = requests.get(url)
# if response.status_code == 200:
#     payrolldata = response.json
#     payrolldata = pd.DataFrame(response.json())
#     payrolldata.to_csv('Citywide_Payroll_Data.csv', index=False)
#     print("Data saved to 'Citywide_Payroll_Data.csv'.")
# else:
#     print("Failed to retrieve data. Status code:", response.status_code)

In [ ]:
# #upload the dataset

# # Specify the path to the JSON configuration file
# config_file_path = 'config.json'

# # Load the JSON configuration file
# with open(config_file_path, 'r') as config_file:
#     config = json.load(config_file)

# connection_str = config["connectionstring"]

# container_name = "citywidesalary"

# blob_name = "Citywide_Payroll_Data_20240318.csv"

# file_path = r"D:\All File\Course\CIS9440\HW1\CIS9440_HW\Citywide_Payroll_Data__Fiscal_Year__20240318.csv"
# azure_upload_blob(connection_str, container_name, blob_name, file_path)


In [7]:
# Specify the path to the JSON configuration file
config_file_path = 'config.json'

# Load the JSON configuration file
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

# Retrieve the connection string from the configuration file
# Make sure that the key matches exactly what's in the config file
CONNECTION_STRING_AZURE_STORAGE = config["connectionstring"]
CONTAINER_NAME = 'citywidesalary'  # The name of the container we want to access

# Initialize the BlobServiceClient with the provided connection string
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

# Get the client for the specified container
container_client = blob_service_client.get_container_client(CONTAINER_NAME)

citywide_salary_df = pd.DataFrame()

# List all blobs in the specified container
blob_list = container_client.list_blobs()
for blob in blob_list:
    print(blob.name)  # Print the name of each blob
    blob_client = container_client.get_blob_client(blob=blob.name)
    blob_data = blob_client.download_blob()
    blob_content = blob_data.readall().decode('utf-8')
    df = pd.read_csv(StringIO(blob_content))  # Load the CSV data into DataFrame
    print(df.shape)  # Display the shape of the DataFrame
    citywide_salary_df = df.copy()  # Copy the DataFrame to the main DataFrame



Citywide_Payroll_Data_20240318.csv


C:\Users\a5119\AppData\Local\Temp\ipykernel_17908\3393227872.py:28: DtypeWarning: Columns (7,10,12,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(StringIO(blob_content))  # Load the CSV data into DataFrame


(5662713, 17)


In [8]:
citywide_salary_df

,Fiscal Year,Payroll Number,Agency Name,Last Name,First Name,Mid Init,Agency Start Date,Work Location Borough,Title Description,Leave Status as of June 30,Base Salary,Pay Basis,Regular Hours,Regular Gross Paid,OT Hours,Total OT Paid,Total Other Pay
0,2020,17.0,OFFICE OF EMERGENCY MANAGEMENT,BEREZIN,MIKHAIL,NaN,08/10/2015,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,"86,005.00",per Annum,"1,820","84,698.21",0,0.00,0.00
1,2020,17.0,OFFICE OF EMERGENCY MANAGEMENT,GEAGER,VERONICA,M,09/12/2016,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,"86,005.00",per Annum,"1,820","84,698.21",0,0.00,0.00
2,2020,17.0,OFFICE OF EMERGENCY MANAGEMENT,RAMANI,SHRADDHA,NaN,02/22/2016,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,"86,005.00",per Annum,"1,820","84,698.21",0,0.00,0.00
3,2020,17.0,OFFICE OF EMERGENCY MANAGEMENT,ROTTA,JONATHAN,D,09/16/2013,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,"86,005.00",per Annum,"1,820","84,698.21",0,0.00,0.00
4,2020,17.0,OFFICE OF EMERGENCY MANAGEMENT,WILSON II,ROBERT,P,04/30/2018,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,"86,005.00",per Annum,"1,820","84,698.21",0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5662708,2023,996.0,NYC HOUSING AUTHORITY,ZUNIGA,JUAN,NaN,10/02/2017,QUEENS,PAINTER,ACTIVE,315.07,per Day,"1,820","81,918.20","1,149","77,575.47","3,752.01"
5662709,2023,996.0,NYC HOUSING AUTHORITY,ZUNIGA,LUIS,NaN,08/21/2017,BRONX,PAINTER,ACTIVE,315.07,per Day,"1,820","81,918.20","1,037.75","70,063.81","3,206.96"
5662710,2023,996.0,NYC HOUSING AUTHORITY,ZUNIGA,PAUL,E,10/07/2019,QUEENS,SUPERVISOR PAINTER,ACTIVE,360.08,per Day,"1,820","93,620.80",908,"70,035.56",617.28
5662711,2023,996.0,NYC HOUSING AUTHORITY,ZUNIGA BERMEO,HUGO,E,07/06/2022,QUEENS,PAINTER,ACTIVE,315.07,per Day,"1,771","79,082.62","1,005.75","67,869.89",0.00


In [18]:
#create a copy
citywide_salary_copy = citywide_salary_df.copy(deep=True)
citywide_salary_copy

,Fiscal Year,Payroll Number,Agency Name,Last Name,First Name,Mid Init,Agency Start Date,Work Location Borough,Title Description,Leave Status as of June 30,Base Salary,Pay Basis,Regular Hours,Regular Gross Paid,OT Hours,Total OT Paid,Total Other Pay
0,2020,17.0,OFFICE OF EMERGENCY MANAGEMENT,BEREZIN,MIKHAIL,NaN,08/10/2015,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,"86,005.00",per Annum,"1,820","84,698.21",0,0.00,0.00
1,2020,17.0,OFFICE OF EMERGENCY MANAGEMENT,GEAGER,VERONICA,M,09/12/2016,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,"86,005.00",per Annum,"1,820","84,698.21",0,0.00,0.00
2,2020,17.0,OFFICE OF EMERGENCY MANAGEMENT,RAMANI,SHRADDHA,NaN,02/22/2016,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,"86,005.00",per Annum,"1,820","84,698.21",0,0.00,0.00
3,2020,17.0,OFFICE OF EMERGENCY MANAGEMENT,ROTTA,JONATHAN,D,09/16/2013,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,"86,005.00",per Annum,"1,820","84,698.21",0,0.00,0.00
4,2020,17.0,OFFICE OF EMERGENCY MANAGEMENT,WILSON II,ROBERT,P,04/30/2018,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,"86,005.00",per Annum,"1,820","84,698.21",0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5662708,2023,996.0,NYC HOUSING AUTHORITY,ZUNIGA,JUAN,NaN,10/02/2017,QUEENS,PAINTER,ACTIVE,315.07,per Day,"1,820","81,918.20","1,149","77,575.47","3,752.01"
5662709,2023,996.0,NYC HOUSING AUTHORITY,ZUNIGA,LUIS,NaN,08/21/2017,BRONX,PAINTER,ACTIVE,315.07,per Day,"1,820","81,918.20","1,037.75","70,063.81","3,206.96"
5662710,2023,996.0,NYC HOUSING AUTHORITY,ZUNIGA,PAUL,E,10/07/2019,QUEENS,SUPERVISOR PAINTER,ACTIVE,360.08,per Day,"1,820","93,620.80",908,"70,035.56",617.28
5662711,2023,996.0,NYC HOUSING AUTHORITY,ZUNIGA BERMEO,HUGO,E,07/06/2022,QUEENS,PAINTER,ACTIVE,315.07,per Day,"1,771","79,082.62","1,005.75","67,869.89",0.00


#### Clean & Transform Data

In [19]:
# Drop Payroll Number, Last Name, First Name, Mid Init
columns_to_drop = ['Payroll Number', 'Last Name', 'First Name', 'Mid Init']
citywide_salary_copy.drop(columns=columns_to_drop, inplace=True)
citywide_salary_copy

,Fiscal Year,Agency Name,Agency Start Date,Work Location Borough,Title Description,Leave Status as of June 30,Base Salary,Pay Basis,Regular Hours,Regular Gross Paid,OT Hours,Total OT Paid,Total Other Pay
0,2020,OFFICE OF EMERGENCY MANAGEMENT,08/10/2015,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,"86,005.00",per Annum,"1,820","84,698.21",0,0.00,0.00
1,2020,OFFICE OF EMERGENCY MANAGEMENT,09/12/2016,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,"86,005.00",per Annum,"1,820","84,698.21",0,0.00,0.00
2,2020,OFFICE OF EMERGENCY MANAGEMENT,02/22/2016,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,"86,005.00",per Annum,"1,820","84,698.21",0,0.00,0.00
3,2020,OFFICE OF EMERGENCY MANAGEMENT,09/16/2013,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,"86,005.00",per Annum,"1,820","84,698.21",0,0.00,0.00
4,2020,OFFICE OF EMERGENCY MANAGEMENT,04/30/2018,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,"86,005.00",per Annum,"1,820","84,698.21",0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5662708,2023,NYC HOUSING AUTHORITY,10/02/2017,QUEENS,PAINTER,ACTIVE,315.07,per Day,"1,820","81,918.20","1,149","77,575.47","3,752.01"
5662709,2023,NYC HOUSING AUTHORITY,08/21/2017,BRONX,PAINTER,ACTIVE,315.07,per Day,"1,820","81,918.20","1,037.75","70,063.81","3,206.96"
5662710,2023,NYC HOUSING AUTHORITY,10/07/2019,QUEENS,SUPERVISOR PAINTER,ACTIVE,360.08,per Day,"1,820","93,620.80",908,"70,035.56",617.28
5662711,2023,NYC HOUSING AUTHORITY,07/06/2022,QUEENS,PAINTER,ACTIVE,315.07,per Day,"1,771","79,082.62","1,005.75","67,869.89",0.00


In [22]:
#Drop NA and remove duplicated
citywide_salary_copy = citywide_salary_copy.drop_duplicates()
citywide_salary_copy = citywide_salary_copy.dropna()
citywide_salary_copy

,Fiscal Year,Agency Name,Agency Start Date,Work Location Borough,Title Description,Leave Status as of June 30,Base Salary,Pay Basis,Regular Hours,Regular Gross Paid,OT Hours,Total OT Paid,Total Other Pay
0,2020,OFFICE OF EMERGENCY MANAGEMENT,08/10/2015,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,"86,005.00",per Annum,"1,820","84,698.21",0,0.00,0.00
1,2020,OFFICE OF EMERGENCY MANAGEMENT,09/12/2016,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,"86,005.00",per Annum,"1,820","84,698.21",0,0.00,0.00
2,2020,OFFICE OF EMERGENCY MANAGEMENT,02/22/2016,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,"86,005.00",per Annum,"1,820","84,698.21",0,0.00,0.00
3,2020,OFFICE OF EMERGENCY MANAGEMENT,09/16/2013,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,"86,005.00",per Annum,"1,820","84,698.21",0,0.00,0.00
4,2020,OFFICE OF EMERGENCY MANAGEMENT,04/30/2018,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,"86,005.00",per Annum,"1,820","84,698.21",0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5662708,2023,NYC HOUSING AUTHORITY,10/02/2017,QUEENS,PAINTER,ACTIVE,315.07,per Day,"1,820","81,918.20","1,149","77,575.47","3,752.01"
5662709,2023,NYC HOUSING AUTHORITY,08/21/2017,BRONX,PAINTER,ACTIVE,315.07,per Day,"1,820","81,918.20","1,037.75","70,063.81","3,206.96"
5662710,2023,NYC HOUSING AUTHORITY,10/07/2019,QUEENS,SUPERVISOR PAINTER,ACTIVE,360.08,per Day,"1,820","93,620.80",908,"70,035.56",617.28
5662711,2023,NYC HOUSING AUTHORITY,07/06/2022,QUEENS,PAINTER,ACTIVE,315.07,per Day,"1,771","79,082.62","1,005.75","67,869.89",0.00


In [24]:
#remaping name
rename_mapping = {
    'Fiscal Year' : 'fiscal_year',
    'Agency Name' : 'agency_name',
    'Agency Start Date' : 'start_date',
    'Work Location Borough' : 'borough',
    'Title Description' : 'title_description',
    'Leave Status as of June 30' : 'leave_status',
    'Base Salary' : 'base_salary',
    'Pay Basis' : 'pay_basis',
    'Regular Hours' : 'regular_hrs',
    'Regular Gross Paid' : 'regular_gross_paid',
    'OT Hours' : 'ot_hrs',
    'Total OT Paid' : 'total_ot_paid',
    'Total Other Pay' : 'total_other_pay'
}

citywide_salary_copy = citywide_salary_copy.rename(columns=rename_mapping)

In [25]:
# there is a 9999 year in the start_data (str) So I need to remove it.
mask = citywide_salary_copy['start_date'].str.contains('9999')
citywide_salary_copy = citywide_salary_copy[~mask]
citywide_salary_copy

,fiscal_year,agency_name,start_date,borough,title_description,leave_status,base_salary,pay_basis,regular_hrs,regular_gross_paid,ot_hrs,total_ot_paid,total_other_pay
0,2020,OFFICE OF EMERGENCY MANAGEMENT,08/10/2015,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,"86,005.00",per Annum,"1,820","84,698.21",0,0.00,0.00
1,2020,OFFICE OF EMERGENCY MANAGEMENT,09/12/2016,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,"86,005.00",per Annum,"1,820","84,698.21",0,0.00,0.00
2,2020,OFFICE OF EMERGENCY MANAGEMENT,02/22/2016,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,"86,005.00",per Annum,"1,820","84,698.21",0,0.00,0.00
3,2020,OFFICE OF EMERGENCY MANAGEMENT,09/16/2013,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,"86,005.00",per Annum,"1,820","84,698.21",0,0.00,0.00
4,2020,OFFICE OF EMERGENCY MANAGEMENT,04/30/2018,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,"86,005.00",per Annum,"1,820","84,698.21",0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5662708,2023,NYC HOUSING AUTHORITY,10/02/2017,QUEENS,PAINTER,ACTIVE,315.07,per Day,"1,820","81,918.20","1,149","77,575.47","3,752.01"
5662709,2023,NYC HOUSING AUTHORITY,08/21/2017,BRONX,PAINTER,ACTIVE,315.07,per Day,"1,820","81,918.20","1,037.75","70,063.81","3,206.96"
5662710,2023,NYC HOUSING AUTHORITY,10/07/2019,QUEENS,SUPERVISOR PAINTER,ACTIVE,360.08,per Day,"1,820","93,620.80",908,"70,035.56",617.28
5662711,2023,NYC HOUSING AUTHORITY,07/06/2022,QUEENS,PAINTER,ACTIVE,315.07,per Day,"1,771","79,082.62","1,005.75","67,869.89",0.00


In [26]:
#drop value < 0 and upper case all str
for column in citywide_salary_copy.columns:
    if citywide_salary_copy[column].dtype == 'float64':
        citywide_salary_copy = citywide_salary_copy[citywide_salary_copy[column] >= 0]

for column in citywide_salary_copy.columns:
    if citywide_salary_copy[column].dtype == 'object':
        citywide_salary_copy[column] = citywide_salary_copy[column].str.upper()

citywide_salary_copy


,fiscal_year,agency_name,start_date,borough,title_description,leave_status,base_salary,pay_basis,regular_hrs,regular_gross_paid,ot_hrs,total_ot_paid,total_other_pay
0,2020,OFFICE OF EMERGENCY MANAGEMENT,08/10/2015,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,"86,005.00",PER ANNUM,"1,820","84,698.21",0,0.00,0.00
1,2020,OFFICE OF EMERGENCY MANAGEMENT,09/12/2016,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,"86,005.00",PER ANNUM,"1,820","84,698.21",0,0.00,0.00
2,2020,OFFICE OF EMERGENCY MANAGEMENT,02/22/2016,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,"86,005.00",PER ANNUM,"1,820","84,698.21",0,0.00,0.00
3,2020,OFFICE OF EMERGENCY MANAGEMENT,09/16/2013,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,"86,005.00",PER ANNUM,"1,820","84,698.21",0,0.00,0.00
4,2020,OFFICE OF EMERGENCY MANAGEMENT,04/30/2018,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,"86,005.00",PER ANNUM,"1,820","84,698.21",0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5662708,2023,NYC HOUSING AUTHORITY,10/02/2017,QUEENS,PAINTER,ACTIVE,315.07,PER DAY,"1,820","81,918.20","1,149","77,575.47","3,752.01"
5662709,2023,NYC HOUSING AUTHORITY,08/21/2017,BRONX,PAINTER,ACTIVE,315.07,PER DAY,"1,820","81,918.20","1,037.75","70,063.81","3,206.96"
5662710,2023,NYC HOUSING AUTHORITY,10/07/2019,QUEENS,SUPERVISOR PAINTER,ACTIVE,360.08,PER DAY,"1,820","93,620.80",908,"70,035.56",617.28
5662711,2023,NYC HOUSING AUTHORITY,07/06/2022,QUEENS,PAINTER,ACTIVE,315.07,PER DAY,"1,771","79,082.62","1,005.75","67,869.89",0.00


In [27]:
#Transform data type
citywide_salary_copy['fiscal_year'] = citywide_salary_copy['fiscal_year'].astype(int)
citywide_salary_copy['agency_name'] = citywide_salary_copy['agency_name'].astype(str)
citywide_salary_copy['start_date'] = pd.to_datetime(citywide_salary_copy['start_date'])
citywide_salary_copy['borough'] = citywide_salary_copy['borough'].astype(str)
citywide_salary_copy['title_description'] = citywide_salary_copy['title_description'].astype(str)
citywide_salary_copy['leave_status'] = citywide_salary_copy['leave_status'].astype(str)
citywide_salary_copy['base_salary'] = citywide_salary_copy['base_salary'].replace('[\$,]', '', regex=True).astype(float)
citywide_salary_copy['pay_basis'] = citywide_salary_copy['pay_basis'].astype(str)
citywide_salary_copy['regular_hrs'] = citywide_salary_copy['regular_hrs'].replace('[\$,]', '', regex=True).astype(float)
citywide_salary_copy['regular_gross_paid'] = citywide_salary_copy['regular_gross_paid'].replace('[\$,]', '', regex=True).astype(float)
citywide_salary_copy['ot_hrs'] = citywide_salary_copy['ot_hrs'].replace('[\$,]', '', regex=True).astype(float)
citywide_salary_copy['total_ot_paid'] = citywide_salary_copy['total_ot_paid'].replace('[\$,]', '', regex=True).astype(float)
citywide_salary_copy['total_other_pay'] = citywide_salary_copy['total_other_pay'].replace('[\$,]', '', regex=True).astype(float)
citywide_salary_copy

,fiscal_year,agency_name,start_date,borough,title_description,leave_status,base_salary,pay_basis,regular_hrs,regular_gross_paid,ot_hrs,total_ot_paid,total_other_pay
0,2020,OFFICE OF EMERGENCY MANAGEMENT,2015-08-10,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00
1,2020,OFFICE OF EMERGENCY MANAGEMENT,2016-09-12,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00
2,2020,OFFICE OF EMERGENCY MANAGEMENT,2016-02-22,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00
3,2020,OFFICE OF EMERGENCY MANAGEMENT,2013-09-16,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00
4,2020,OFFICE OF EMERGENCY MANAGEMENT,2018-04-30,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5662708,2023,NYC HOUSING AUTHORITY,2017-10-02,QUEENS,PAINTER,ACTIVE,315.07,PER DAY,1820.0,81918.20,1149.00,77575.47,3752.01
5662709,2023,NYC HOUSING AUTHORITY,2017-08-21,BRONX,PAINTER,ACTIVE,315.07,PER DAY,1820.0,81918.20,1037.75,70063.81,3206.96
5662710,2023,NYC HOUSING AUTHORITY,2019-10-07,QUEENS,SUPERVISOR PAINTER,ACTIVE,360.08,PER DAY,1820.0,93620.80,908.00,70035.56,617.28
5662711,2023,NYC HOUSING AUTHORITY,2022-07-06,QUEENS,PAINTER,ACTIVE,315.07,PER DAY,1771.0,79082.62,1005.75,67869.89,0.00


In [28]:
#check data type
citywide_salary_copy.dtypes

fiscal_year                    int32
agency_name                   object
start_date            datetime64[ns]
borough                       object
title_description             object
leave_status                  object
base_salary                  float64
pay_basis                     object
regular_hrs                  float64
regular_gross_paid           float64
ot_hrs                       float64
total_ot_paid                float64
total_other_pay              float64
dtype: object

In [29]:
citywide_salary_copy.head()

,fiscal_year,agency_name,start_date,borough,title_description,leave_status,base_salary,pay_basis,regular_hrs,regular_gross_paid,ot_hrs,total_ot_paid,total_other_pay
0,2020,OFFICE OF EMERGENCY MANAGEMENT,2015-08-10,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.0,PER ANNUM,1820.0,84698.21,0.0,0.0,0.0
1,2020,OFFICE OF EMERGENCY MANAGEMENT,2016-09-12,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.0,PER ANNUM,1820.0,84698.21,0.0,0.0,0.0
2,2020,OFFICE OF EMERGENCY MANAGEMENT,2016-02-22,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.0,PER ANNUM,1820.0,84698.21,0.0,0.0,0.0
3,2020,OFFICE OF EMERGENCY MANAGEMENT,2013-09-16,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.0,PER ANNUM,1820.0,84698.21,0.0,0.0,0.0
4,2020,OFFICE OF EMERGENCY MANAGEMENT,2018-04-30,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.0,PER ANNUM,1820.0,84698.21,0.0,0.0,0.0


In [30]:
#year 2024 not yet end, so remove all start_date within 2024
date_threshold = pd.Timestamp('2023-12-31')
citywide_salary_copy = citywide_salary_copy[citywide_salary_copy['start_date'] <= date_threshold]
citywide_salary_copy

,fiscal_year,agency_name,start_date,borough,title_description,leave_status,base_salary,pay_basis,regular_hrs,regular_gross_paid,ot_hrs,total_ot_paid,total_other_pay
0,2020,OFFICE OF EMERGENCY MANAGEMENT,2015-08-10,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00
1,2020,OFFICE OF EMERGENCY MANAGEMENT,2016-09-12,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00
2,2020,OFFICE OF EMERGENCY MANAGEMENT,2016-02-22,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00
3,2020,OFFICE OF EMERGENCY MANAGEMENT,2013-09-16,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00
4,2020,OFFICE OF EMERGENCY MANAGEMENT,2018-04-30,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5662708,2023,NYC HOUSING AUTHORITY,2017-10-02,QUEENS,PAINTER,ACTIVE,315.07,PER DAY,1820.0,81918.20,1149.00,77575.47,3752.01
5662709,2023,NYC HOUSING AUTHORITY,2017-08-21,BRONX,PAINTER,ACTIVE,315.07,PER DAY,1820.0,81918.20,1037.75,70063.81,3206.96
5662710,2023,NYC HOUSING AUTHORITY,2019-10-07,QUEENS,SUPERVISOR PAINTER,ACTIVE,360.08,PER DAY,1820.0,93620.80,908.00,70035.56,617.28
5662711,2023,NYC HOUSING AUTHORITY,2022-07-06,QUEENS,PAINTER,ACTIVE,315.07,PER DAY,1771.0,79082.62,1005.75,67869.89,0.00


In [19]:
# #drop the year that not need
# year_to_drop = list(range(2014,2018))
# condition = citywide_salary_copy['fiscal_year'].isin(year_to_drop)
# citywide_salary_copy = citywide_salary_copy[~condition]
# citywide_salary_copy


#### Combin the dataset with dimension table

In [31]:
citywide_salary_clean = citywide_salary_copy.copy(deep=True)
citywide_salary_clean


,fiscal_year,agency_name,start_date,borough,title_description,leave_status,base_salary,pay_basis,regular_hrs,regular_gross_paid,ot_hrs,total_ot_paid,total_other_pay
0,2020,OFFICE OF EMERGENCY MANAGEMENT,2015-08-10,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00
1,2020,OFFICE OF EMERGENCY MANAGEMENT,2016-09-12,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00
2,2020,OFFICE OF EMERGENCY MANAGEMENT,2016-02-22,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00
3,2020,OFFICE OF EMERGENCY MANAGEMENT,2013-09-16,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00
4,2020,OFFICE OF EMERGENCY MANAGEMENT,2018-04-30,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5662708,2023,NYC HOUSING AUTHORITY,2017-10-02,QUEENS,PAINTER,ACTIVE,315.07,PER DAY,1820.0,81918.20,1149.00,77575.47,3752.01
5662709,2023,NYC HOUSING AUTHORITY,2017-08-21,BRONX,PAINTER,ACTIVE,315.07,PER DAY,1820.0,81918.20,1037.75,70063.81,3206.96
5662710,2023,NYC HOUSING AUTHORITY,2019-10-07,QUEENS,SUPERVISOR PAINTER,ACTIVE,360.08,PER DAY,1820.0,93620.80,908.00,70035.56,617.28
5662711,2023,NYC HOUSING AUTHORITY,2022-07-06,QUEENS,PAINTER,ACTIVE,315.07,PER DAY,1771.0,79082.62,1005.75,67869.89,0.00


In [32]:
citywide_salary_clean.dtypes

fiscal_year                    int32
agency_name                   object
start_date            datetime64[ns]
borough                       object
title_description             object
leave_status                  object
base_salary                  float64
pay_basis                     object
regular_hrs                  float64
regular_gross_paid           float64
ot_hrs                       float64
total_ot_paid                float64
total_other_pay              float64
dtype: object

In [34]:
# Fiscal Year Dimension Table
df_fiscal_year = pd.DataFrame({'fiscal_year_id': range(1, len(citywide_salary_clean['fiscal_year'].unique()) + 1),
                               'fiscal_year': sorted(citywide_salary_clean['fiscal_year'].unique())})

df_fiscal_year

,fiscal_year_id,fiscal_year
0,1,2014
1,2,2015
2,3,2016
3,4,2017
4,5,2018
5,6,2019
6,7,2020
7,8,2021
8,9,2022
9,10,2023


In [35]:
# Agency Dimension Table
df_agency = pd.DataFrame({'agency_id': range(1, len(citywide_salary_clean['agency_name'].str.upper().unique()) + 1),
                          'agency_name': sorted(citywide_salary_clean['agency_name'].str.upper().unique())})
df_agency

,agency_id,agency_name
0,1,ADMIN FOR CHILDREN'S SVCS
1,2,ADMIN TRIALS AND HEARINGS
2,3,BOARD OF CORRECTION
3,4,BOARD OF ELECTION
4,5,BOARD OF ELECTION POLL WORKERS
...,...,...
158,159,STATEN ISLAND COMMUNITY BD #3
159,160,TAX COMMISSION
160,161,TAXI & LIMOUSINE COMMISSION
161,162,TEACHERS RETIREMENT SYSTEM


In [36]:
# # Start Date Dimension Table
# df_start_date = pd.DataFrame({'start_date_id': range(1, len(citywide_salary_clean['start_date'].unique()) + 1),
#                               'start_date': sorted(citywide_salary_clean['start_date'].unique())})
# df_start_date['date_iso_format'] = df_start_date['start_date'].dt.strftime('%Y-%m-%dT%H:%M:%S')
# df_start_date['year_number'] = df_start_date['start_date'].dt.year
# df_start_date['quarter_number'] = df_start_date['start_date'].dt.quarter
# df_start_date['month_number'] = df_start_date['start_date'].dt.month
# df_start_date['day_number'] = df_start_date['start_date'].dt.day
# df_start_date['hour_number'] = df_start_date['start_date'].dt.hour
# df_start_date['month_name'] = df_start_date['start_date'].dt.strftime('%B')
# df_start_date['day_name'] = df_start_date['start_date'].dt.strftime('%A')
# df_start_date['week_of_year'] = df_start_date['start_date'].dt.isocalendar().week
# df_start_date['week_of_month'] = df_start_date['start_date'].dt.day.apply(lambda x: (x - 1) // 7 + 1)
# new_order = ['start_date_id', 'start_date', 'date_iso_format', 'year_number', 'quarter_number', 'month_number', 'day_number', 'hour_number', 'month_name', 'day_name', 'week_of_year', 'week_of_month']
# df_start_date = df_start_date[new_order]
# df_start_date


df_start_date = pd.DataFrame({
    'start_date': sorted(citywide_salary_clean['start_date'].unique())
})

df_start_date['start_date_id'] = df_start_date['start_date'].dt.strftime('%Y%m%d').astype(int)
df_start_date['date_iso_format'] = df_start_date['start_date'].dt.strftime('%Y-%m-%dT%H:%M:%S')
df_start_date['year_number'] = df_start_date['start_date'].dt.year
df_start_date['quarter_number'] = df_start_date['start_date'].dt.quarter
df_start_date['month_number'] = df_start_date['start_date'].dt.month
df_start_date['day_number'] = df_start_date['start_date'].dt.day
df_start_date['hour_number'] = df_start_date['start_date'].dt.hour
df_start_date['month_name'] = df_start_date['start_date'].dt.strftime('%B')
df_start_date['day_name'] = df_start_date['start_date'].dt.strftime('%A')
df_start_date['week_of_year'] = df_start_date['start_date'].dt.isocalendar().week
df_start_date['week_of_month'] = df_start_date['start_date'].dt.day.apply(lambda x: (x - 1) // 7 + 1)

new_order = ['start_date_id', 'start_date', 'date_iso_format', 'year_number', 'quarter_number', 'month_number', 'day_number', 'hour_number', 'month_name', 'day_name', 'week_of_year', 'week_of_month']
df_start_date = df_start_date[new_order]

df_start_date


,start_date_id,start_date,date_iso_format,year_number,quarter_number,month_number,day_number,hour_number,month_name,day_name,week_of_year,week_of_month
0,19010101,1901-01-01,1901-01-01T00:00:00,1901,1,1,1,0,January,Tuesday,1,1
1,19450902,1945-09-02,1945-09-02T00:00:00,1945,3,9,2,0,September,Sunday,35,1
2,19460325,1946-03-25,1946-03-25T00:00:00,1946,1,3,25,0,March,Monday,13,4
3,19480901,1948-09-01,1948-09-01T00:00:00,1948,3,9,1,0,September,Wednesday,36,1
4,19480916,1948-09-16,1948-09-16T00:00:00,1948,3,9,16,0,September,Thursday,38,3
...,...,...,...,...,...,...,...,...,...,...,...,...
15930,20230623,2023-06-23,2023-06-23T00:00:00,2023,2,6,23,0,June,Friday,25,4
15931,20230625,2023-06-25,2023-06-25T00:00:00,2023,2,6,25,0,June,Sunday,25,4
15932,20230626,2023-06-26,2023-06-26T00:00:00,2023,2,6,26,0,June,Monday,26,4
15933,20230627,2023-06-27,2023-06-27T00:00:00,2023,2,6,27,0,June,Tuesday,26,4


In [37]:
# Borough Dimension Table
unique_boroughs = citywide_salary_clean['borough'].str.upper().unique()
nyc_boroughs = ['MANHATTAN', 'BROOKLYN', 'QUEENS', 'BRONX', 'RICHMOND']
filtered_boroughs = [borough for borough in unique_boroughs if borough in nyc_boroughs]
df_borough = pd.DataFrame({'borough_id': range(1, len(filtered_boroughs) + 1),
                           'borough': sorted(filtered_boroughs)})
df_borough['borough_id']=df_borough['borough_id'].astype('Int64')

df_borough

,borough_id,borough
0,1,BRONX
1,2,BROOKLYN
2,3,MANHATTAN
3,4,QUEENS
4,5,RICHMOND


In [38]:
df_borough.dtypes

borough_id     Int64
borough       object
dtype: object

In [39]:
# Create title descriptions table

citywide_salary_clean['title_description'] = citywide_salary_clean['title_description'].str.replace('*', '').str.strip().str.upper()
citywide_salary_clean['title_description'] = citywide_salary_clean['title_description'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x).upper())
df_title_description = pd.DataFrame({'title_description_id': range(1, len(citywide_salary_clean['title_description'].unique()) + 1),
                                     'title_description': sorted(citywide_salary_clean['title_description'].unique())})
df_title_description


,title_description_id,title_description
0,1,12 MONTH SPECIAL EDUCATION ASST PRINCIPAL
1,2,12 MONTH SPECIAL EDUCATION SUPERVISOR
2,3,ACCOUNTANT
3,4,ACTUARIAL SPECIALIST LEVEL I
4,5,ACTUARIAL SPECIALIST LEVEL II OJ
...,...,...
1844,1845,WIPER
1845,1846,WORKERS COMPENSATION BENEFITS EXAMINER
1846,1847,XRAY TECHNICIAN
1847,1848,YOUTH COORDINATOR


In [40]:
# Leave Status Dimension Table
df_leave_status = pd.DataFrame({'leave_status_id': range(1, len(citywide_salary_clean['leave_status'].str.upper().unique()) + 1),
                                'leave_status': sorted(citywide_salary_clean['leave_status'].str.upper().unique())})
df_leave_status

,leave_status_id,leave_status
0,1,ACTIVE
1,2,CEASED
2,3,ON LEAVE
3,4,ON SEPARATION LEAVE
4,5,SEASONAL


In [41]:
# Pay Basis Dimension Table
df_pay_basis = pd.DataFrame({'pay_basis_id': range(1, len(citywide_salary_clean['pay_basis'].str.upper().unique()) + 1),
                             'pay_basis': sorted(citywide_salary_clean['pay_basis'].str.upper().unique())})
df_pay_basis

,pay_basis_id,pay_basis
0,1,PER ANNUM
1,2,PER DAY
2,3,PER HOUR
3,4,PRORATED ANNUAL


In [42]:
# Create Fact table
# Add fiscal year foreign key
fact_table = pd.merge(citywide_salary_clean, df_fiscal_year, left_on='fiscal_year', right_on='fiscal_year', how='left')

# Add agency foreign key
fact_table = pd.merge(fact_table, df_agency, left_on='agency_name', right_on='agency_name', how='left')

# Add start date foreign key
fact_table = pd.merge(fact_table, df_start_date, left_on='start_date', right_on='start_date', how='left')

# Add borough foreign key
fact_table = pd.merge(fact_table, df_borough, left_on='borough', right_on='borough', how='left')

# Add title description foreign key
fact_table = pd.merge(fact_table, df_title_description, left_on='title_description', right_on='title_description', how='left')

# Add leave status foreign key
fact_table = pd.merge(fact_table, df_leave_status, left_on='leave_status', right_on='leave_status', how='left')

# Add pay basis foreign key
fact_table = pd.merge(fact_table, df_pay_basis, left_on='pay_basis', right_on='pay_basis', how='left')

# Add 'fact_id' column before creating the fact table
fact_table['fact_id'] = range(1, len(fact_table) + 1)

# Create the fact table
fact_table = fact_table[['fact_id', 'fiscal_year_id', 'agency_id', 'start_date_id', 'borough_id', 'title_description_id', 'leave_status_id', 'pay_basis_id', 'base_salary', 'regular_hrs', 'regular_gross_paid', 'ot_hrs', 'total_ot_paid', 'total_other_pay']]

fact_table


,fact_id,fiscal_year_id,agency_id,start_date_id,borough_id,title_description_id,leave_status_id,pay_basis_id,base_salary,regular_hrs,regular_gross_paid,ot_hrs,total_ot_paid,total_other_pay
0,1,7,125,20150810,2,944,1,1,86005.00,1820.0,84698.21,0.00,0.00,0.00
1,2,7,125,20160912,2,944,1,1,86005.00,1820.0,84698.21,0.00,0.00,0.00
2,3,7,125,20160222,2,944,1,1,86005.00,1820.0,84698.21,0.00,0.00,0.00
3,4,7,125,20130916,2,944,1,1,86005.00,1820.0,84698.21,0.00,0.00,0.00
4,5,7,125,20180430,2,944,1,1,86005.00,1820.0,84698.21,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4668656,4668657,10,121,20171002,4,1303,1,2,315.07,1820.0,81918.20,1149.00,77575.47,3752.01
4668657,4668658,10,121,20170821,1,1303,1,2,315.07,1820.0,81918.20,1037.75,70063.81,3206.96
4668658,4668659,10,121,20191007,4,1753,1,2,360.08,1820.0,93620.80,908.00,70035.56,617.28
4668659,4668660,10,121,20220706,4,1303,1,2,315.07,1771.0,79082.62,1005.75,67869.89,0.00


#### Push to DW

In [31]:
from sqlalchemy import create_engine

username = 'Weisui_Xu'
password = 'Cis9440!HW'
host = 'cis9440hw.postgres.database.azure.com'
port = 5432
dbname = 'postgres'

database_url = f'postgresql://{username}:{password}@{host}:{port}/{dbname}?sslmode=require'

engine = create_engine(
    database_url,
    connect_args={'options': '-csearch_path={}'.format('citywidesalary')},
)


In [32]:
df_fiscal_year.to_sql('dim_fiscal_year', engine, if_exists='append', index=False) #1

10

In [33]:
df_agency.to_sql('dim_agency', engine, if_exists='append', index=False) #2


163

In [34]:
df_start_date.to_sql('dim_start_date', engine, if_exists='append', index=False) #3


927

In [35]:
df_borough.to_sql('dim_borough', engine, if_exists='append', index=False) #4


5

In [36]:
df_title_description.to_sql('dim_title_description', engine, if_exists='append', index=False) #5


846

In [37]:
df_leave_status.to_sql('dim_leave_status', engine, if_exists='append', index=False) #6


5

In [38]:
df_pay_basis.to_sql('dim_pay_basis', engine, if_exists='append', index=False) #7


4

In [39]:
fact_table.to_sql('fact_salaries', engine, if_exists='append', index=False) #8

420